Helperfunctions

**xor_two_bit_lists** takes two bitlists *a* and *b* and returns a list with $a_i ^ b_i | 0 < i < min(len(a), len(b))$

**rotate** rotates a bitlist by the **factor** times (positive -> left shift, negative -> right shift)

The last four functions are for converting integers or hexstrings to the internally used bitlists and back

In [3]:
def xor_two_bit_lists(bit_list_a, bit_list_b):
    return map(lambda x: x[0] ^ x[1], zip(bit_list_a, bit_list_b))

def rotate(l, factor):
    return l[factor:] + l[:factor]

def bitlist_to_int(bit_list):
    value = 0
    for bit in bit_list:
        value = (value << 1) | bit
    return value

def int_to_bitlist(integer):
    bitlist = []
    for x in range(4):
        bitlist = [integer & 0x1] + bitlist
        integer = integer >> 1
    return bitlist

def bitlist_to_hexstring(bitlist):
    return str(hex(int("".join(map(lambda x: str(x), bitlist)), base=2))[2:]).zfill(len(bitlist)//4)

def hexstring_to_bitlist(hexstring, resulting_length):
    if hexstring[-1] == 'L': 
        hexstring = hexstring[:-1]
    bits = bin(int(hexstring, base=16))
    bitlist = map(lambda x: int(x), list(bits[2:]))
    bitlist = [0] * (resulting_length-len(bitlist))+ bitlist
    return bitlist

hexstring_to_bitlist("00", 5)


[0, 0, 0, 0, 0]

Reading all sboxes and saving them in the **sbox** list

In [4]:
sbox = []
for x in range(1, 9):
    input_table = [list() for i in range(4)]
    input_file = open("sboxes/sbox%s.txt" % x, "r")
    for i in range(4):
        for line_index in range(16):
            input_table[i].append(int(input_file.readline()))
    sbox.append(input_table)
sbox

[[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
  [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
  [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
  [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],
 [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
  [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
  [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
  [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],
 [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
  [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
  [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
  [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],
 [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
  [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
  [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
  [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],
 [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
  [14, 11,

Definition of the many permutation tables used in the DES Encryption

In [5]:
expansion_table = [32,  1,  2,  3,  4,  5,
                    4,  5,  6,  7,  8,  9,
                    8,  9, 10, 11, 12, 13,
                   12, 13, 14, 15, 16, 17,
                   16, 17, 18, 19, 20, 21,
                   20, 21, 22, 23, 24, 25,
                   24, 25, 26, 27, 28, 29,
                   28, 29, 30, 31, 32,  1]

pc1_table = [57, 49, 41, 33, 25, 17,  9,  1,
             58, 50, 42, 34, 26, 18, 10,  2,
             59, 51, 43, 35, 27, 19, 11,  3,
             60, 52, 44, 36, 63, 55, 47, 39,
             31, 23, 15,  7, 62, 54, 46, 38,
             30, 22, 14,  6, 61, 53, 45, 37,
             29, 21, 13,  5, 28, 20, 12,  4]

pc2_table = [14, 17, 11, 24,  1,  5,  3, 28,
             15,  6, 21, 10, 23, 19, 12,  4,
             26,  8, 16,  7, 27, 20, 13,  2,
             41, 52, 31, 37, 47, 55, 30, 40,
             51, 45, 33, 48, 44, 49, 39, 56,
             34, 53, 46, 42, 50, 36, 29, 32]

permute_table_p = [16,  7, 20, 21, 29, 12, 28, 17,
                    1, 15, 23, 26,  5, 18, 31, 10,
                    2,  8, 24, 14, 32, 27,  3,  9,
                   19, 13, 30,  6, 22, 11,  4, 25]

initial_permutation_table = [58, 50, 42, 34, 26, 18, 10,  2,
                             60, 52, 44, 36, 28, 20, 12,  4,
                             62, 54, 46, 38, 30, 22, 14,  6,
                             64, 56, 48, 40, 32, 24, 16,  8,
                             57, 49, 41, 33, 25, 17,  9,  1,
                             59, 51, 43, 35, 27, 19, 11,  3,
                             61, 53, 45, 37, 29, 21, 13,  5,
                             63, 55, 47, 39, 31, 23, 15,  7]

final_permutation_table = [40, 8, 48, 16, 56, 24, 64, 32,
                           39, 7, 47, 15, 55, 23, 63, 31,
                           38, 6, 46, 14, 54, 22, 62, 30,
                           37, 5, 45, 13, 53, 21, 61, 29,
                           36, 4, 44, 12, 52, 20, 60, 28,
                           35, 3, 43, 11, 51, 19, 59, 27,
                           34, 2, 42, 10, 50, 18, 58, 26,
                           33, 1, 41,  9, 49, 17, 57, 25]

# input is a bit list
def permutation(inputlist, permutation_table):
    result = []
    for x in permutation_table:
        result.append(inputlist[x-1])
    return result

In [42]:
bitlist_to_hexstring(permutation(hexstring_to_bitlist("0xA0B1C0D1E1F0F0F0", 64), pc1_table))

'fffcf3e000000a'

Genertating the round key using input key, which has been stripped from its parity bits (using permutation table **permute_table_p**)

In [8]:
#input_key is a bit list
def gen_key_for_round(input_key, round):
    c_part = input_key[0:28]
    d_part = input_key[28:56]
    for i in range(round):
        if i == 0 or i == 1 or i == 8 or i == 15:
            c_part = rotate(c_part, 1)
            d_part = rotate(d_part, 1)
        else:
            c_part = rotate(c_part, 2)
            d_part = rotate(d_part, 2)
    
    rotated_key = c_part + d_part
    return permutation(rotated_key, pc2_table)
    
print gen_key_for_round([0] * 56, 3)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
def use_sboxes_on_bit_list(bit_list):
    result = []
    for x in range(8):
        value = bitlist_to_int(bit_list[x*6:(x+1)*6])
        middle = (value >> 1) & 15
        ends = ((value >> 5) << 1) + (value & 1)
        result = result + int_to_bitlist(sbox[x][ends][middle])
    return result
        
print use_sboxes_on_bit_list([0, 1] * 24)

[1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]


implements the DES Feistel function, **k** must have a length of 56 (parity bits are stripped -> to do that use the pc1 permutation). The result needs to be permuted by P (**permute_table_p**) for using it further in the DES encryption.

In [10]:
def des_f_function(k, e, round):
    expanded_e = permutation(e, expansion_table)
    generated_key = gen_key_for_round(k, round)
    xored_list = xor_two_bit_lists(expanded_e, generated_key)
    sboxed_list = use_sboxes_on_bit_list(xored_list)
    return sboxed_list

print bitlist_to_hexstring(des_f_function([1] * 56, [1] * 32, 0))
print bitlist_to_hexstring(des_f_function([0] * 56, hexstring_to_bitlist("d8d8dbbc", 8), 1))

efa72c4d
5bb997f3


**des_encrypt** implements a complete encryption of a 64 bit plaintext block with a 56 bit key (see **des_f_function** for further instructions on how to strip the parity bits).

In [45]:
# plaintext is 64 bit long
def des_encrypt(k, plaintext):
    #initial permutation
    l = plaintext[0:32]
    r = plaintext[32:64]
    for x in range(16):
        temp = r
        f_function_result = permutation(des_f_function(k, r, x+1), permute_table_p)
        r = xor_two_bit_lists(l, f_function_result)
        l = temp
    print "DEBUG: result before permutation: %s" % (bitlist_to_hexstring(r+l))
    return permutation(r + l, final_permutation_table)

print bitlist_to_hexstring(des_encrypt([0] * 56, [0] * 64))

DEBUG: result before permutation: 1c2087fcbbea0dc2
8ca64de9c1b123a7L


print the result of **des_f_function** using random values. This was used to verify the correctness of **des_f_function**

In [12]:
from random import randint
input_value = hex(randint(0, 2**64))
r = permutation(hexstring_to_bitlist(input_value, 64), initial_permutation_table)[32:64]
input_key = hex(randint(0, 2**64))
real_key = permutation(hexstring_to_bitlist(input_key, 64), pc1_table)
input_round = randint(1, 16)

print "plaintext e         : %s" % input_value
print "key k               : %s" % input_key
print "key k without parity: 0x%s" % bitlist_to_hexstring(real_key)
print "round               : %d" % input_round
print "r                   : 0x%s" % bitlist_to_hexstring(r)
print "round_key           : 0x%s" % bitlist_to_hexstring(gen_key_for_round(real_key, 1))
print "f_function result   : 0x%s" % bitlist_to_hexstring(permutation(des_f_function(real_key, r, input_round), permute_table_p))

plaintext e         : 0x5e7e6b647772c62a
key k               : 0xc3d7de018e4a0c23L
key k without parity: 0x1727800b756746
round               : 13
r                   : 0x40be87f7
round_key           : 0x86d032ba67f0
f_function result   : 0x394ee15c


print the result of **des_encrypt** using random values. This was used to verify the correctness of **des_encrypt**

In [13]:
from random import randint
input_value = hex(randint(0, 2**64))
lr = permutation(hexstring_to_bitlist(input_value, 64), initial_permutation_table)
input_key = hex(randint(0, 2**64))
real_key = permutation(hexstring_to_bitlist(input_key, 64), pc1_table)
input_round = 1# randint(1, 16)

print "plaintext e         : %s" % input_value
print "key k               : %s" % input_key
print "key k without parity: %s" % bitlist_to_hexstring(real_key)
print "round               : %d" % input_round
print "r                   : %s" % bitlist_to_hexstring(r)
print "round_key           : %s" % bitlist_to_hexstring(gen_key_for_round(real_key, 1))
print "des_encrypt result  : 0x%s" % bitlist_to_hexstring(des_encrypt(real_key, lr))

plaintext e         : 0xec4ce6727a90d635L
key k               : 0x70d0f9eeb37f784b
key k without parity: 1eef7d7b828ec7
round               : 1
r                   : 40be87f7
round_key           : c6ffce275d61
des_encrypt result  : 0xda9976ade110a1c4L


generating and formating of random testcases for the DES Feistel function implementation in the SAT solver.

In [14]:
from random import randint
def f_function_test_gen(number_of_tests):
    rounds = []
    e = []
    k = []
    out = []
    for x in enumerate(range(number_of_tests)):
        input_value = "0x0"#hex(randint(0, 2**64))
        right = permutation(hexstring_to_bitlist(input_value, 64), initial_permutation_table)[32:64]
        input_key = "0x0"#hex(randint(0, 2**64))
        real_key = permutation(hexstring_to_bitlist(input_key, 64), pc1_table)
        input_round = 1#randint(1, 16)
        rounds.append(input_round)
        e.append("0x%s" % bitlist_to_hexstring(right))
        k.append("0x%s" % bitlist_to_hexstring(real_key))
        out.append("0x%s" % bitlist_to_hexstring(des_f_function(real_key, right, input_round)))
    
    print "unsigned r[] = {%s};" % str(rounds)[1:-1]
    print "unsigned e[] = {%s};" % str(e)[1:-1].translate(None, "\'")
    print "uint64_t k[] = {%s};" % str(k)[1:-1].translate(None, "\'")
    print "unsigned out[] = {%s};" % str(out)[1:-1].translate(None, "\'")

f_function_test_gen(1)

unsigned r[] = {1};
unsigned e[] = {0x00000000};
uint64_t k[] = {0x00000000000000};
unsigned out[] = {0xefa72c4d};


In [15]:
from random import randint
def round_test_gen(number_of_tests):
    rounds = []
    l = []
    r = []
    k = []
    out = []
    for x in enumerate(range(number_of_tests)):
        input_value = hex(randint(0, 2**64))
        right = hexstring_to_bitlist(input_value, 64)[32:64]
        left = hexstring_to_bitlist(input_value, 64)[0:32]
        input_key = hex(randint(0, 2**64))
        real_key = permutation(hexstring_to_bitlist(input_key, 64), pc1_table)
        input_round = randint(1, 16)
        rounds.append(input_round)
        l.append("0x%s" % bitlist_to_hexstring(left))
        r.append("0x%s" % bitlist_to_hexstring(right))
        k.append("0x%s" % bitlist_to_hexstring(real_key))
        f_result = des_f_function(real_key, right, input_round)
        permuted_f_result = permutation(f_result, permute_table_p)
        xor_new_r_old_l = xor_two_bit_lists(permuted_f_result, left)
        out.append("0x%s" % bitlist_to_hexstring(xor_new_r_old_l))
    
    print "unsigned rounds[] = {%s};" % str(rounds)[1:-1]
    print "unsigned l[] = {%s};" % str(l)[1:-1].translate(None, "\'")
    print "unsigned r[] = {%s};" % str(r)[1:-1].translate(None, "\'")
    print "uint64_t k[] = {%s};" % str(k)[1:-1].translate(None, "\'")
    print "unsigned out[] = {%s};" % str(out)[1:-1].translate(None, "\'")

round_test_gen(5)

unsigned rounds[] = {10, 6, 11, 8, 4};
unsigned l[] = {0xb6291746, 0x1405437c, 0x67a59285, 0x00d9cd99, 0xecaf329e};
unsigned r[] = {0xec6c759a, 0x6bf68358, 0x616d00ce, 0x410f23f5, 0xc304095e};
uint64_t k[] = {0x2eefe22d74860b, 0xad6cd465dcce83, 0x1ca0a1d5fde457, 0xab8aefbc955c01, 0x83d54cdc04cefb};
unsigned out[] = {0x66b10e70, 0x69409a27, 0xa3babe6b, 0x599c1a70, 0x13206748};


In [16]:
# plaintext is 64 bit long
def des_encrypt_without_permutations(k, plaintext):
    #initial permutation
    l = plaintext[0:32]
    r = plaintext[32:64]
    for x in range(16):
        temp = r
        f_function_result = permutation(des_f_function(k, r, x+1), permute_table_p)
        r = xor_two_bit_lists(l, f_function_result)
        l = temp
    return r + l

print bitlist_to_hexstring(des_encrypt_without_permutations([0] * 56, [0] * 64))

1c2087fcbbea0dc2


In [17]:
from random import randint
def round_test_gen(number_of_tests):
    inputs = []
    k = []
    out = []
    for x in enumerate(range(number_of_tests)):
        input_value = hex(randint(0, 2**64))
        permuted_input = permutation(hexstring_to_bitlist(input_value, 64), initial_permutation_table)
        right = permuted_input[32:64]
        left = permuted_input[0:32]
        input_key = hex(randint(0, 2**64))
        real_key = permutation(hexstring_to_bitlist(input_key, 64), pc1_table)
        inputs.append("0x%s" % bitlist_to_hexstring(left + right))
        k.append("0x%s" % bitlist_to_hexstring(real_key))
        des_encrypt_result = des_encrypt_without_permutations(real_key, left + right)
        out.append("0x%s" % bitlist_to_hexstring(des_encrypt_result))
    
    print "uint64_t input[] = {%s};" % str(inputs)[1:-1].translate(None, "\'")
    print "uint64_t k[] = {%s};" % str(k)[1:-1].translate(None, "\'")
    print "uint64_t out[] = {%s};" % str(out)[1:-1].translate(None, "\'")

round_test_gen(5)

uint64_t input[] = {0x19b31de363f058f4, 0xe6826595bd691e95L, 0x8f07e0d246caa636L, 0x608ae3a006bc2a45, 0xc92a62797ca4d4bfL};
uint64_t k[] = {0xd77fc5b1e11330, 0xdc4eae905c44a8, 0x13201df49fc40c, 0x483a7c77533805, 0xa92b47fbd7ca7b};
uint64_t out[] = {0x94afe37d158470f8L, 0x7ab311bb9e32f378, 0xfdb133beba3e1e6eL, 0xf82535214e78dd08L, 0x205b09b4c3ba0ae2};


In [50]:
from random import randint
input_value = "1337DEADBEEF2015"
lr = permutation(hexstring_to_bitlist(input_value, 64), initial_permutation_table)
input_key = "password".encode('hex')
real_key = permutation(hexstring_to_bitlist(input_key, 64), pc1_table)
input_round = 1# randint(1, 16)

print "plaintext e         : %s" % input_value
print "plaintext e permuted: %s" % bitlist_to_hexstring(lr)
print "key k               : %s" % input_key
print "key k without parity: %s" % bitlist_to_hexstring(real_key)
print "round               : %d" % input_round
print "round_key           : %s" % bitlist_to_hexstring(gen_key_for_round(real_key, 1))
print "des_encrypt result  : 0x%s" % bitlist_to_hexstring(des_encrypt(real_key, lr))
print "ruby result         : 0x70d80826b159ee30"

plaintext e         : 1337DEADBEEF2015
plaintext e permuted: 2497beab3c7a3c37
key k               : 70617373776f7264
key k without parity: 00ffff57cb020d
round               : 1
round_key           : e0be6e662267
DEBUG: result before permutation: 06149dfadc96f486
des_encrypt result  : 0x0463fe85bd0989af
ruby result         : 0x70d80826b159ee30


In [44]:
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("F0F0F0F0F0F0F0F0", 64), pc1_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("AAAAAAAAAAAAAAAA", 64), initial_permutation_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("0000000000000001", 64), initial_permutation_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("0000000000000002", 64), initial_permutation_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("0000000000000001", 64), pc1_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("0000000000000002", 64), pc1_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("KeorM`rr".encode('hex'), 64), pc1_table))
print bitlist_to_hexstring(permutation(hexstring_to_bitlist("JensLars".encode('hex'), 64), pc1_table))
print "KeorM`rr".encode('hex')
print "JensLars".encode('hex')
print hexstring_to_bitlist("0000000000000001", 64)[0]

fffffff000000f
00000000ffffffff
0000008000000000
0000000000000080
00000000000000
00000008000000
00ffeeccd16158
00ffeeccd16158
4b656f724d607272
4a656e734c617273
0
